In [2]:
import os
import cv2
import numpy as np
import mediapipe as mp

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

In [4]:
IMAGE_FILES = []
emotion_key ={
    0: 'none',
    1: 'happy',
    2: 'sadness',
    3: 'angry',
    4: 'disgust',
    5: 'surprise',
    6: 'fear',
    7: 'neutral',
    'happy': 1,
    'sadness': 2,
    'angry': 3,
    'disgust': 4,
    'surprise': 5,
    'fear': 6,
    'none': 0,
    'neutral': 7,
}


CORDS_EMOTION_PATH = os.path.join('data', 'cords_emotion.npy')
INDEX_EMOTION_PATH = os.path.join('data', 'index_emotion.npy')
IMAGES_EMOTION_ID_PATH = os.path.join('data', 'images')
EDIT_IMAGES_EMOTION_ID_PATH = os.path.join('data', 'edit')

arr_cords_emotion = np.zeros((1, 478, 3))
arr_index_emotion = np.zeros(1)
np.save(CORDS_EMOTION_PATH, arr_cords_emotion)
np.save(INDEX_EMOTION_PATH, arr_index_emotion)

In [5]:
def EmotionCordsArray(emotion_id, face_landmarks):
    global arr_cords_emotion
    
    arr_cords = list()
    for item in face_landmarks.landmark:
        arr_cords.append([item.x, item.y, item.z])
    
    arr_cords = np.array(arr_cords)
    
    with open(CORDS_EMOTION_PATH, 'rb') as file:
        arr_cords_emotion = np.load(file, allow_pickle=True)
    with open(INDEX_EMOTION_PATH, 'rb') as file:
        arr_index_emotion = np.load(file, allow_pickle=True)
    
    with open(CORDS_EMOTION_PATH, 'wb') as file:
        arr_cords_emotion = np.append(arr_cords_emotion, [arr_cords], axis=0)
        np.save(file, arr_cords_emotion)
    with open(INDEX_EMOTION_PATH, 'wb') as file:
        arr_index_emotion = np.append(arr_index_emotion, emotion_id)
        np.save(file, arr_index_emotion)

    del arr_cords

In [6]:
with mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True) as face_mesh:
    for emotion in os.listdir(IMAGES_EMOTION_ID_PATH):
        if emotion == '.DS_Store':
            continue
            
        for index, file in enumerate(os.listdir(f'{IMAGES_EMOTION_ID_PATH}/{emotion}')):

            image = cv2.imread(f'{IMAGES_EMOTION_ID_PATH}/{emotion}/{file}')
            results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

            if results.multi_face_landmarks:
                face_landmarks = results.multi_face_landmarks[0]
                mp_drawing.draw_landmarks(image, face_landmarks, mp_face_mesh.FACEMESH_TESSELATION, None,
                                      mp_drawing_styles.get_default_face_mesh_tesselation_style())

#                 cv2.imwrite(f'{EDIT_IMAGES_EMOTION_ID_PATH}/{index}.png', image)
                EmotionCordsArray(emotion_key[emotion], face_landmarks)
            if index % 100 == 0:
                print(index)
        print(f'EMOTION: {emotion}, Completed')
    print('COMPELTED')

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400


KeyboardInterrupt: 

In [49]:
ff = np.zeros((2, 6, 3))
d = [[1,2,3],[1,2,3],[1,2,3],[1,2,3],[1,2,3],[1,2,3]]
ff = np.append(ff, [d], axis=0)
print(ff)

[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[1. 2. 3.]
  [1. 2. 3.]
  [1. 2. 3.]
  [1. 2. 3.]
  [1. 2. 3.]
  [1. 2. 3.]]]


In [ ]:
emotion_id = int(input())
while emotion_id not in [0,1,2,3,4,5,6]:
    emotion_id = int(input())

cap = cv2.VideoCapture(0)
while True:
    ret, image = cap.read()
    
    if cv2.waitKey(5) & 0xFF == 121: # exit on ESC
        os.listdir(IMAGES_EMOTION_ID_PATH)[-1]
        cv2.imwrite(f'{IMAGES_EMOTION_ID_PATH}/{str((os.listdir(IMAGES_EMOTION_ID_PATH)[-1]) + 1)}.png', image)
        print('Good')
        break
    
    with mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True) as face_mesh:
        image.flags.writeable = False
        results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        image.flags.writeable = True
        
        if results.multi_face_landmarks:
            face_landmarks = results.multi_face_landmarks[0]
            mp_drawing.draw_landmarks(image, face_landmarks, mp_face_mesh.FACEMESH_TESSELATION, None,
                                      mp_drawing_styles.get_default_face_mesh_tesselation_style())

    if cv2.waitKey(5) & 0xFF == 27: # exit on ESC
        break

cap.release()
cv2.destroyAllWindows()

In [146]:
# cap = cv2.VideoCapture(0)
# for imgnum in range(number_images):
#     print(f'Image: {imgnum}')
#     idimg = str(uuid.uuid4())
            
#     ret, image = cap.read()
    
#     imgname = os.path.join(IMAGES_PATH, f'{idimg}.jpg')
#     cv2.imwrite(imgname, image)
            
#     with mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True) as face_mesh:
#         image.flags.writeable = False
#         # Меняем цветовую гамму
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#         # Либа по посторойке координат на лице
#         results = face_mesh.process(image)
#         image.flags.writeable = True
#         # Возвращаем цвет обратно
#         image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
    
#         if results.multi_face_landmarks:
#             face_landmarks = results.multi_face_landmarks[0]
#             mp_drawing.draw_landmarks(image, face_landmarks, mp_face_mesh.FACEMESH_TESSELATION, None,
#                                       mp_drawing_styles.get_default_face_mesh_tesselation_style())

#         dictMediapipeInArray(idimg, emotion_id, face_landmarks)

#     imgname = os.path.join(EDIT_IMAGES_PATH, f'{idimg}.jpg')
#     cv2.imwrite(imgname, image)
#     time.sleep(0.5)

#     cv2.imshow('image', image)

#     if cv2.waitKey(5) & 0xFF == 27: # exit on ESC
#         break

# cap.release()
# cv2.destroyAllWindows()

In [26]:
# Импорт необходимых модулей
import cv2
import numpy as np
import dlib

# Запуск видеопотока
cap = cv2.VideoCapture(0)

# Подключение детектора, настроеного на поиск человеческих лиц
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("./shape_predictor_68_face_landmarks.dat")

while True:
    
    # Получение изображения из видеопотока
    ret, image = cap.read()

    # Конвертирование изображения в черно-белое
    grayFrame = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Обнаружение лиц и построение прямоугольного контура
    faces = detector(grayFrame)

    # Обход списка всех лиц попавших на изображение
    for face in faces:

        # Выводим количество лиц на изображении
        cv2.putText(image, "{} face(s) found".format(len(faces)), (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # Получение координат вершин прямоугольника и его построение на изображении
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.bottom()
        cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 1)
        
        # Получение координат контрольных точек и их построение на изображении
        landmarks = predictor(grayFrame, face)
        for n in range(0, 68):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            cv2.circle(image, (x, y), 3, (255, 0, 0), -1)

    cv2.putText(image, "Press ESC to close frame", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Вывод преобразованного изображения
    cv2.imshow("Frame", image)

    # Для выхода из цикла нажать ESC
    key = cv2.waitKey(1)
    if key == 27:
        break
